<a href="https://colab.research.google.com/github/Justholli/labs/blob/main/lab2_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Возьмите небольшой (3-5 предложений) текстовый отрывок из вашего
любимого фильма, книги или стихотворения на английском языке.
Можно взять на русском и перевести

Озвучьте его голосом на ваш вкус с помощью двух API сервисов
https://zvukogram.com/ (API docs https://zvukogram.com/node/api/)
https://elevenlabs.io/ (API docs https://docs.elevenlabs.io/welcome/introduction)

Эксперименты с разными голосами и языками приветствуются, можете воспользоваться python + requests (либо любой язык на ваш выбор). Пользоваться графическим интерфейсом категорически НЕ рекомендуется.

Затем вам нужно озвучить тот же фрагмент с помощью модели Tortoise TTS. https://github.com/neonbjb/tortoise-tts

Можете для начальных тестов взять один из готовых голосов в репозитории, но конечная цель - получить похожий на любого персонажа голос, поместив собственные семплы в папку voices

In [8]:
text_to_speech = "Bait is poison, fed up with grandiose ideas; they say start with yourself — I killed the empire within me. The master turns crimson: they can't evict us. Our flag has white snow and a blue river (and that's all). The stage is sick, war is near. They dance the cancan for a piece of shit pie. You get bubble gum, I go back to the underground."

Api сервис для озвучки выбрал elevenlabs

**Установим все зависимости**

In [1]:
!pip3 install elevenlabs
!pip3 install -U scipy
!pip3 install -r requirements.txt
!pip3 install transformers==4.19.0 einops==0.5.0 rotary_embedding_torch==0.1.5 unidecode==1.3.5

!git clone https://github.com/jnordberg/tortoise-tts.git
%cd tortoise-tts
!python3 setup.py install

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
fatal: destination path 'tortoise-tts' already exists and is not an empty directory.
/content/tortoise-tts
running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ***********************************

In [3]:
import torch
import torchaudio
import os
import IPython
import torch.nn as nn
import torch.nn.functional as F

from elevenlabs.client import ElevenLabs
from elevenlabs import play, stream, save
from google.colab import files
from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices


In [16]:
client = ElevenLabs(api_key="5b9bb9b4efb8ef5619c92bf04525029b")

In [17]:
audioFromApi = client.generate(
  text=text_to_speech,
  voice="Mimi",
  model="eleven_multilingual_v2"
)

save(audioFromApi, "/content/sample_data/my-file.mp3")

In [18]:
IPython.display.Audio("/content/sample_data/my-file.mp3")

In [19]:
audioFromApi = client.generate(
  text=text_to_speech,
  voice="Joseph",
  model="eleven_multilingual_v2"
)
save(audioFromApi, "/content/sample_data/my-file-men.mp3")

In [20]:
IPython.display.Audio("/content/sample_data/my-file-men.mp3")

In [4]:
tts = TextToSpeech()

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [6]:
preset = "high_quality"

# Название нового голоса
CUSTOM_VOICE_NAME = "lab_test"

# Загрузка семплов голоса. WAV формат
custom_voice_folder = f"/content/tortoise-tts/tortoise/voices/{CUSTOM_VOICE_NAME}"
os.makedirs(custom_voice_folder)
for i, file_data in enumerate(files.upload().values()):
  with open(os.path.join(custom_voice_folder, f'{i}.wav'), 'wb') as f:
    f.write(file_data)

Saving 1 (1).wav to 1 (1) (1).wav


In [9]:
voice_samples, conditioning_latents = load_voice(CUSTOM_VOICE_NAME)
gen = tts.tts_with_preset(text_to_speech, voice_samples=voice_samples, conditioning_latents=conditioning_latents,
                          preset=preset)
torchaudio.save(f'generated-{CUSTOM_VOICE_NAME}.wav', gen.squeeze(0).cpu(), 24000)
IPython.display.Audio(f'generated-{CUSTOM_VOICE_NAME}.wav')

Generating autoregressive samples..


100%|██████████| 16/16 [05:28<00:00, 20.53s/it]


Computing best candidates using CLVP and CVVP


  0%|          | 0/16 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
  6%|▋         | 1/16 [00:02<00:38,  2.54s/it]

No stop tokens found in one of the generated voice clips. This typically means the spoken audio is too long. In some cases, the output will still be good, though. Listen to it and if it is missing words, try breaking up your input text.


...

 88%|████████▊ | 14/16 [00:20<00:02,  1.42s/it]

No stop tokens found in one of the generated voice clips. This typically means the spoken audio is too long. In some cases, the output will still be good, though. Listen to it and if it is missing words, try breaking up your input text.


100%|██████████| 16/16 [00:23<00:00,  1.48s/it]


Transforming autoregressive outputs into audio..


100%|██████████| 400/400 [06:55<00:00,  1.04s/it]
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:303: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(F.pad(input, self._reversed_padding_repeated_twice, mode=self.padding_mode),
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:797: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv_transpose1d(


In [10]:
IPython.display.Audio(f'generated-{CUSTOM_VOICE_NAME}.wav')

ElevenLabs предлагает высокое качество синтеза речи с естественным произношением и эмоциональной выразительностью, создавая очень естественный голос. Этот сервис прост в использовании и предоставляет готовое решение с предустановленными настройками. Tortoise TTS также генерирует качественную речь, и, хотя могут встречаться небольшие артефакты, он превосходит в гибкости интонаций и возможности точной настройки эмоциональной окраски, позволяя добиваться более детализированных интонационных вариаций.